## JSON 파일 병합 

In [1]:
import json

# 파일 경로
file1 = "/Users/gim-yujin/Desktop/pjt_personal_agent/영양소 데이터/JSON 파일 /supplement_rag_data_fixed.json"         # 1차 파싱 성공 214개
file2 = "/Users/gim-yujin/Desktop/pjt_personal_agent/영양소 데이터/JSON 파일 /supplements_rag_data_final8.json"       # 8차 보완 파싱 결과

# JSON 로드
with open(file1, "r", encoding="utf-8") as f1, open(file2, "r", encoding="utf-8") as f2:
    data1 = json.load(f1)
    data2 = json.load(f2)

# 병합
merged_data = data1 + data2

# 저장
output_file = "supplement_rag_data_merged.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=2)

print(f"📦 병합된 JSON 저장 완료: {output_file}")
print(f"✅ 총 레코드 수: {len(merged_data)}개")

📦 병합된 JSON 저장 완료: supplement_rag_data_merged.json
✅ 총 레코드 수: 306개


- 리뷰데이터 포함한 json 파일 생성
	•	"title" 기준으로 정확히 일치할 경우에만 평점/리뷰를 붙입니다.
	•	일치하지 않는 경우에는 "avg_rating": null, "reviews_count": null 으로 남습니다.
	•	"Avg rating" 값은 문자열로 저장되어 있을 수 있어, 후속 처리 시 float 타입으로 변환할 수 있습니다 (예: float(item["avg_rating"])).

In [3]:
import json
import pandas as pd
from tqdm import tqdm

# 1️⃣ 파일 경로 설정
merged_file = "/Users/gim-yujin/Desktop/pjt_personal_agent/영양소 데이터/JSON 파일 /supplement_rag_data_merged_cleaned.json"
raw_iherb_file = "/Users/gim-yujin/Desktop/pjt_personal_agent/영양소 데이터/JSON 파일 /iherb_data_uk_data_2022_12.json"
output_file = "supplement_rag_data_merged_with_reviews.json"

# 2️⃣ 원본 JSON 로드
with open(raw_iherb_file, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# 3️⃣ 제목 → 평점/리뷰수 딕셔너리 생성
rating_dict = {}
for item in raw_data:
    title = item.get("Title", "").strip()
    rating = item.get("Avg rating", None)
    reviews = item.get("Reviews count", None)

    if title:
        rating_dict[title] = {
            "Avg rating": rating,
            "Reviews count": reviews
        }

# 4️⃣ 기존 RAG JSON 로드
with open(merged_file, "r", encoding="utf-8") as f:
    rag_data = json.load(f)

# 5️⃣ 병합: title 기준
for item in tqdm(rag_data):
    title = item.get("title", "").strip()
    rating_info = rating_dict.get(title, {})
    item["avg_rating"] = rating_info.get("Avg rating", None)
    item["reviews_count"] = rating_info.get("Reviews count", None)

# 6️⃣ 저장
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(rag_data, f, ensure_ascii=False, indent=2)

print(f"✅ 새로운 JSON 저장 완료: {output_file}")

100%|██████████| 306/306 [00:00<00:00, 381073.94it/s]

✅ 새로운 JSON 저장 완료: supplement_rag_data_merged_with_reviews.json
